In [1]:
pip install beir

Note: you may need to restart the kernel to use updated packages.


In [2]:
from beir import util
from beir.datasets.data_loader import GenericDataLoader

data_path = util.download_and_unzip(
    "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/scifact.zip",
    "./datasets"
)

corpus, queries, qrels = GenericDataLoader(data_path).load(split="test")

/home/kroybal/miniconda3/envs/cs197-rag/lib/python3.12/site-packages/beir/util.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


  0%|          | 0/5183 [00:00<?, ?it/s]

In [9]:
# pip install rank-bm25 nltk
from rank_bm25 import BM25Okapi
import re
import numpy as np

# --- simple tokenizer (good enough for BEIR baselines) ---
token_pat = re.compile(r"\w+")

def tokenize(text: str):
    return token_pat.findall(text.lower())

# --- 1) flatten corpus into aligned lists ---
doc_ids = list(corpus.keys())
doc_texts = [
    (corpus[doc_id].get("title","") + " " + corpus[doc_id].get("text","")).strip()
    for doc_id in doc_ids
]
tokenized_docs = [tokenize(text) for text in doc_texts]

In [10]:
# --- 2) build BM25 index ---
bm25 = BM25Okapi(tokenized_docs)

In [11]:
# --- 3) retrieval function ---
def bm25_retrieve(query_text: str, k: int = 10):
    q_tokens = tokenize(query_text)
    scores = bm25.get_scores(q_tokens)  # numpy array, len = #docs
    top_idx = scores.argsort()[::-1][:k]
    return [(doc_ids[i], float(scores[i])) for i in top_idx]

In [22]:
# --- 4) run BM25 on the whole query set ---
k = 10
bm25_results = {}  # qid -> list of (doc_id, score)
for qid, qtext in queries.items():
    bm25_results[qid] = bm25_retrieve(qtext, k=k)

In [23]:
# Example: show top-5 for one query
some_qid = next(iter(queries))
print("Query:", queries[some_qid])
print("Top docs:", bm25_results[some_qid][:5])

Query: 0-dimensional biomaterials show inductive properties.
Top docs: [('10608397', 10.173663486906072), ('43385013', 9.863533386166488), ('40212412', 9.556998622831742), ('10931595', 9.112927676965947), ('27049238', 8.914352928774871)]


In [42]:
#if at least one rel doc is retrieved, bm25 succeeds
def is_hit_at_k(retrieved_doc_ids, qrel_dict):
    gold = set(qrel_dict.keys())  # relevant doc ids
    return any(d in gold for d in retrieved_doc_ids)

hits = []
not_hit = []
qid_categories = {}
for qid in queries:
    retrieved = [doc_id for doc_id, _ in bm25_results[qid]]
    hit = is_hit_at_k(retrieved, qrels.get(qid, {}))
    if not hit:
        not_hit.append(qid)
        qid_categories[qid] = {
            "bm25": False,
            "query": qtext,
            "Failure Category": None  # fill manually
        }
    else:
        qid_categories[qid] = {
            "bm25": True,
            "query": qtext,
            "Failure Category": "not a failure"
        }
    hits.append(hit)
print(f"BM25 hit@{k}: {sum(hits)}/{len(hits)} = {sum(hits)/len(hits):.3f}")

BM25 hit@10: 239/300 = 0.797


In [50]:
label_map = {
    "Paraphrase": {
        1, 13, 132, 238, 239, 384, 623, 800, 870, 913, 914,
        975, 1049, 1088, 1110, 1191, 1199, 1213, 1226, 1241,
        1278, 1279, 1316, 1332, 1368
    },
    "Entities": {
        431, 437, 517, 535, 544, 577, 690, 715, 716, 768,
        775, 830, 831, 1200, 1280, 1281, 1382
    },
    "Keyword overload": {
        502, 820, 821, 887, 1196, 1197, 783, 1395
    },
    "Rare terms": {
        1140, 1221
    }
}

for qid in qid_categories:
    if not qid_categories[qid]["bm25"]:  # only label failures
        for label, qids in label_map.items():
            qid_categories[qid]["Failure Category"] = label
            break


In [53]:
qid_categories

{'1': {'bm25': False,
  'query': 'p16INK4A accumulation is  linked to an abnormal wound response caused by the microinvasive step of advanced Oral Potentially Malignant Lesions (OPMLs).',
  'Failure Category': 'Paraphrase'},
 '3': {'bm25': True,
  'query': 'p16INK4A accumulation is  linked to an abnormal wound response caused by the microinvasive step of advanced Oral Potentially Malignant Lesions (OPMLs).',
  'Failure Category': 'not a failure'},
 '5': {'bm25': True,
  'query': 'p16INK4A accumulation is  linked to an abnormal wound response caused by the microinvasive step of advanced Oral Potentially Malignant Lesions (OPMLs).',
  'Failure Category': 'not a failure'},
 '13': {'bm25': False,
  'query': 'p16INK4A accumulation is  linked to an abnormal wound response caused by the microinvasive step of advanced Oral Potentially Malignant Lesions (OPMLs).',
  'Failure Category': 'Paraphrase'},
 '36': {'bm25': True,
  'query': 'p16INK4A accumulation is  linked to an abnormal wound respons

In [26]:
#Categorize failure modes (rare terms, paraphrases, entities, keyword overload)
not_hit_dict = {}

for qid in not_hit:
    retrieved = [doc_id for doc_id, _ in bm25_results[qid]]
    true_relevant = qrels.get(qid, {})

    not_hit_dict[qid] = {
        "retrieved": retrieved,
        "true_relevant": true_relevant
    }

# for qid in not_hit:
#     print(f"QID: {qid}")
#     print("Query:", queries[qid])
#     print("-" * 60)

In [27]:
# for qid in not_hit:
#     print(f"QID: {qid}")
#     print("Query:", queries[qid])
#     print()

#     print("Retrieved by BM25 (titles):")
#     for doc_id, score in bm25_results[qid]:
#         title = corpus[doc_id].get("title", "")
#         print(f"  - {title}  (score={score:.2f})")

#     print("\nRelevant (gold) documents (titles):")
#     for doc_id in qrels.get(qid, {}):
#         title = corpus[doc_id].get("title", "")
#         print(f"  - {title}")

#     print("=" * 80)

Paraphrases (semantic match, weak exact overlap)

BM25 fails because the gold doc expresses the same idea with different wording (synonyms, reformulations, broader framing).

QID 1 — Paraphrase: “0-dimensional biomaterials” ↔ “nanotechnologies”; BM25 latches onto generic “properties” and returns unrelated “properties” papers.

QID 13 — Paraphrase: “low birth weight” ↔ “childhood underweight”; numbers/phrasing don’t overlap cleanly with the gold title.

QID 132 — Paraphrase: claim about aspirin→PGE2; gold discusses COX/immune evasion framing rather than “PGE2” wording.

QID 238 — Paraphrase: query ties methionine restriction to miRNAs; gold is a general microRNA concept title, not matching “methionine restriction”.

QID 239 — Paraphrase: “older appearance” ↔ “perceived age as biomarker”; BM25 matches “aging” docs but misses “perceived age” phrasing.

QID 384 — Paraphrase: “low economic settings” ↔ risk assessment framing; gold title doesn’t mirror the socioeconomic phrasing.

QID 623 — Paraphrase: “low vitamin D” ↔ “past sun exposure/skin phenotype”; gold uses proxy variables rather than “vitamin D”.

QID 800 — Paraphrase: query is about epigenome + aging + neurogenesis; gold title is broader “neurological disease” framing.

QID 870 — Paraphrase: “life quality” ↔ mortality; BM25 matches “quality of life” papers but gold is BMI–mortality.

QID 913 / 914 — Paraphrase: PPAR ligands effects framed broadly as “nuclear hormone receptor allosteric pathways” in gold.

QID 975 — Paraphrase: query is mechanistic (“primary cytokines induce secondary mediators”); gold is variability/genetics of cytokine response.

QID 1049 — Paraphrase: “ribosomopathies… tissue-specific pathology” ↔ “ribosome-mediated specificity in Hox translation”.

QID 1088 — Paraphrase: query says “silencing BCL2”; gold says “BCL-2 required for maintenance” (opposite framing, different surface form).

QID 1110 — Paraphrase: “suboptimal nutrition” ↔ large comparative risk assessment framing in gold.

QID 1191 — Paraphrase: query about growth rate of public DNA; gold is a specific database (“EMBL nucleotide sequence database”).

QID 1199 — Paraphrase: query about colchicine benefits + statins; gold is broader determinants of event rates, not that exact phrasing.

QID 1213 — Paraphrase: query about monocyte activation harms; gold is specific KLF2 inactivation/atherosclerosis.

QID 1226 — Paraphrase: query links TET loss to myeloid cancers; gold focuses on Tet1/2 roles in ESC lineage specification.

QID 1241 — Paraphrase: query about mesodermal cardiac progenitors; gold uses “KDR+ population” wording.

QID 1278 / 1279 — Paraphrase: query says “co-IR blockade” + autoimmune events; gold says “autoimmunity Achilles’ heel of immunotherapy”.

QID 1316 — Paraphrase: “UCB T cells memory-like phenotype” ↔ “T-cell repertoire recovery after HSCT”.

QID 1332 — Paraphrase: cytokine interaction claim vs gold about genetic variability of cytokine responses.

QID 1368 — Paraphrase: “term of delivery” vs gold “pregnancy and neonatal outcomes” systematic review/meta-analysis.

Entities (exact named thing matters; BM25 retrieves “nearby” but not the specific gold)

BM25 fails because the query hinges on a specific gene/protein/drug/strain/dataset entity, and the gold uses a different entity name or higher-level grouping.

QID 431 — Entities: “FoxO3a” vs gold “FOXO transcription factors / SIRT1”; BM25 overweights “ROS” and misses the FOXO/SIRT1 angle.

QID 437 — Entities: query about MDS; gold is about glioma iPSC models (entity/topic mismatch).

QID 517 — Entities: query about copeptin; gold is BNP Mendelian randomization.

QID 535 — Entities: query about type 1 diabetes + hypertension; gold is broader “glucose tolerance and blood pressure” (population mismatch).

QID 544 — Entities: query about IFIT1; gold is about RIG-I methylation tolerance.

QID 577 — Entities: query about P. chabaudi inoculum dynamics; gold uses different framing (“effective propagation number”) without the same surface entity cues.

QID 690 — Entities: query about SFM syndrome + lactate; gold is volume depletion in malaria (topic/entity mismatch).

QID 715 / 716 — Entities: query about miR7a; gold about ceRNA hypothesis (different key entity).

QID 768 — Entities: query about TPMT metabolism; gold about NT5C2 resistance in ALL.

QID 775 — Entities: query “DNA polI (polymerase I)” vs gold polymerases μ/λ/TdT (different polymerase entity).

QID 830 / 831 — Entities: query focuses on NF2 (Merlin); gold focuses on LKB1–MARK axis controlling Hippo–YAP.

QID 1200 — Entities: query about ML-SA1 at hTRPML1/2; gold is TRPML1 structures (missing ML-SA1/TRPML2 cues).

QID 1280 / 1281 — Entities: query about ure gene cluster; gold is urea channel structure (channel ≠ gene cluster proteins).

QID 1382 — Entities: “aPKCz/PKCζ” naming mismatch; BM25 may not connect symbol variants reliably.

Keyword overload / ambiguity (BM25 matches the wrong sense because too many generic words)

BM25 fails because the query includes ambiguous or overloaded keywords (classic example: “delivery” or “N-terminal”), so it matches many irrelevant docs that share the common words.

QID 502 — Keyword overload/ambiguity: “delivery centers” triggers many “drug delivery”/generic “delivery” papers; BM25 chases the wrong sense of delivery.

QID 820 / 821 — Keyword overload/ambiguity: “N-terminal” is overloaded; BM25 matches “start sites / cleavage / N-terminal” documents, but gold is about N-terminal acetylation proteomics.

QID 887 — Keyword overload/ambiguity: “spores” matches Bacillus anthracis spores etc.; gold is a specific “cooperative bacterium antagonism” framing.

QID 1196 / 1197 — Keyword overload/ambiguity: “safe places to study” has little lexical overlap with gold (“mental health problems…”); BM25 keys on “homelessness” and retrieves generic homelessness papers.

QID 783 — Keyword overload: query contains many specific induction details (EAM, α-MyHC/CFA, IFN-γ receptor); gold frames it as IL-12/IFN-γ axis—BM25 focuses on the extra tokens that don’t appear in gold.

QID 1395 — Keyword overload: query is very long with niche terms (OPMLs, microinvasive, wound response); gold is the p16INK4a luciferase model—BM25 gets distracted by the rare extra tokens.

Rare terms (the query uses a rare synonym/chemical/form; gold uses a common name)

BM25 fails when the rare term isn’t literally present in the gold title (even though it’s the same concept).

QID 1140 — Rare terms: “α-tocopheryl acetate” is a rare/formal name; gold uses “Vitamins E and C…”. If “tocopheryl acetate” doesn’t appear in the title, BM25 won’t connect it to “vitamin E”.

QID 1221 — Rare terms / spelling: query has “matasteses” (misspelling of metastases). BM25 is brittle to typos; if the gold uses “metastasis/metastases” correctly, the mismatch can kill overlap.

Quick pattern you can write in your report

Paraphrase failures: gold expresses the claim with different scientific terminology or a proxy variable (sun exposure vs vitamin D; perceived age vs appearance).

Entity failures: query hinges on a specific biomolecule/drug/strain; gold uses a different entity name or umbrella category (IFIT1 vs RIG-I; copeptin vs BNP).

Keyword overload: ambiguous common terms (delivery, N-terminal) dominate scoring and drag BM25 to the wrong sense.

Rare-term failures: rare synonyms/chemical names/typos break exact overlap.